In [20]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, AutoModelForSequenceClassification, AutoTokenizer
from transformers import BertTokenizerFast, EncoderDecoderModel
import torch
from transformers import pipeline
import matplotlib.pyplot as plt

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

model_name_translate = "Helsinki-NLP/opus-mt-es-en"
tokenizer_translate = MarianTokenizer.from_pretrained(model_name_translate)
model_translate = MarianMTModel.from_pretrained(model_name_translate).to(device)

model_name_classify = "facebook/bart-large-mnli"
tokenizer_classify = AutoTokenizer.from_pretrained(model_name_classify)
model_classify = AutoModelForSequenceClassification.from_pretrained(model_name_classify).to(device)

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [11]:
def generate_summary(text):
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False).to(device)
    return summary[0]['summary_text']

def translate_text(text):
    translated = tokenizer_translate(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    translated_text = model_translate.generate(**translated)
    return tokenizer_translate.decode(translated_text[0], skip_special_tokens=True)

def classify_text(text, labels):
    inputs = tokenizer_classify(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        logits = model_classify(**inputs).logits
    scores = torch.softmax(logits, dim=1)
    results = sorted([(label, score.item()) for label, score in zip(labels, scores[0])], key=lambda x: x[1], reverse=True)  # Ajustado para incluir el primer score
    return results[0][0]

In [5]:
df = pd.read_parquet('clean_data.parquet')
first = df.iloc[0]

In [31]:
labels = ["Legislation and Regulation", "Public Administration and Procedures", "Education and Culture", "Economy and Finance"]

text_es = first['text']
# summary_es = generate_summary(text_es)
text_en = translate_text(text_es)
classification_label = classify_text(text_en, labels)


In [30]:
print(text_es)

BOE.es - BOA-d-1991-90001 Decreto Legislativo 1/1991, de 19 de febrero de la Diputación General de Aragón, por el que se aprueba el texto refundido de la Ley de Ordenación de la Función Pública de la Comunidad Autónoma de Aragón.
BOA-d-1991-90001
Documento BOA-d-1991-90001
Decreto Legislativo 1/1991, de 19 de febrero de la Diputación General de Aragón, por el que se aprueba el texto refundido de la Ley de Ordenación de la Función Pública de la Comunidad Autónoma de Aragón.Ver texto consolidado
Publicado en:
«BOA» núm. 25, de 1 de marzo de 1991, páginas 667 a 677 (11 págs.)
Departamento:
Comunidad Autónoma de Aragón
Referencia:
BOA-d-1991-90001
Otros formatos:
Texto
TEXTO ORIGINAL
Por Ley 2/1991, de 4 de enero, de las Cortes de Aragón, se modifican numerosos preceptos de la Ley 1/1986, de 20 de febrero, de Medidas para la Ordenación de la Función Pública de la Comunidad Autónoma de Aragón, aprobando la nueva redacción de aquéllos por los motivos que se expresan en el preámbulo de la cit

In [32]:
print(classification_label)

Legislation and Regulation


In [33]:
first100 = df.head(100)

classification_results = []

for index, row in first100.iterrows():
    text_es = row['text']
    # summary_es = generate_summary(text_es)
    text_en = translate_text(text_es)
    classification_label = classify_text(text_en, labels)
    classification_results.append(classification_label)

In [34]:
print(text_es)

BOE.es - BOC-j-2017-90500 Corrección de errores de la Ley 4/2017, de 13 de julio, del Suelo y de los Espacios Naturales Protegidos de Canarias.
BOC-j-2017-90500
Documento BOC-j-2017-90500
Corrección de errores de la Ley 4/2017, de 13 de julio, del Suelo y de los Espacios Naturales Protegidos de Canarias.
Publicado en:
«BOC» núm. 223, de 20 de noviembre de 2017, páginas 31063 a 31064 (2 págs.)
Departamento:
Comunidad Autónoma de Canarias
Referencia:
BOC-j-2017-90500
Otros formatos:
Texto
TEXTO ORIGINAL
Advertidos errores en el texto de la Ley 4/2017, de 13 de julio, del Suelo y de los Espacios Naturales Protegidos de Canarias (BOC n.º 138, de 19.7.17), se procede a su oportuna rectificación en los siguientes términos:
PREÁMBULO
XVI. Modificación de otras leyes.
Donde dice:
«[...]; y finalmente modifica la Ley 14/2013, de 8 de abril, de Puertos de Canarias, para cambiar la clasificación del puerto de Puerto de la Cruz, en la isla de Tenerife».
Debe decir:
«[...]; y finalmente modifica la

In [35]:
print(classification_results[-1])

Public Administration and Procedures
